# Experiments Notebook

## Global variables & Imports

In [ ]:
# Path to DICOM directory
DICOM_PATH = "/home/pyuser/data/Paradise_DICOMs"
# DICOM_PATH = "/home/pyuser/data/Paradise_Test_DICOMs"

# Path to Histogram directory
HISTOGRAM_PATH = "/home/pyuser/data/Paradise_Histograms"

# Path to Experiments directory
EXPERIMENTS_PATH = "/home/pyuser/data/Paradise_Experiments"

import os
import numpy as np
import pydicom
from PIL import Image
from pathlib import Path
import warnings

# Suppress the specific pydicom character encoding warning
warnings.filterwarnings("ignore", category=UserWarning, module="pydicom.charset")

FILES_TO_CONVERT = [28695112]

## RGB Encoding for Extended Dynamic Range

### How it works


The `convert_dicom_to_rgb_png()` function implements a novel approach to preserve DICOM's high dynamic range in standard PNG format by encoding pixel values across RGB channels.

#### How RGB Encoding Works:

1. **Normalization**: Pixel values are normalized to 0-1 range based on actual data range
2. **Scaling**: Values are scaled to 0-765 range (255 × 3 channels)  
3. **Distribution**: Values are distributed across RGB channels sequentially:
   - **0-255**: All in Red channel `(value, 0, 0)`
   - **256-510**: Red maxed, remainder in Green `(255, value-255, 0)`
   - **511-765**: Red and Green maxed, remainder in Blue `(255, 255, value-510)`

#### Example:
- **Original pixel**: 3000 (in 0-4095 range)
- **Normalized**: 3000/4095 = 0.732
- **Scaled**: 0.732 × 765 = 560
- **RGB encoding**: `(255, 255, 50)` where 255+255+50 = 560

#### Benefits:
- **3× Dynamic Range**: 766 values vs standard grayscale's 256
- **Lossless**: Perfect reconstruction possible via `decode_rgb_png_to_pixels()`
- **Standard Format**: Uses regular PNG files, compatible with any image viewer
- **MONOCHROME Handling**: Automatic conversion between MONOCHROME1 and MONOCHROME2


### Functions

In [ ]:
def convert_dicom_to_rgb_png(files_to_convert, target_monochrome="MONOCHROME2"):
    """
    Convert DICOM files to PNG with RGB encoding for extended dynamic range.
    
    This function encodes pixel values across RGB channels to effectively triple
    the dynamic range compared to standard grayscale conversion.
    
    Parameters:
    -----------
    files_to_convert : list
        List of FileIDs (without extension) to convert
    target_monochrome : str, optional
        Target monochrome format ("MONOCHROME1" or "MONOCHROME2"). Default is "MONOCHROME2"
    
    Returns:
    --------
    dict : Dictionary with conversion results and statistics
    """
    
    # Create experiments directory if it doesn't exist
    Path(EXPERIMENTS_PATH).mkdir(parents=True, exist_ok=True)
    
    # Get all DICOM files in the directory
    all_dicom_files = {}
    for root, dirs, files in os.walk(DICOM_PATH):
        for file in files:
            if file.lower().endswith(('.dcm', '.dicom')):
                # Extract FileID (filename without extension)
                file_id = os.path.splitext(file)[0]
                try:
                    file_id = int(file_id)  # Convert to int if it's numeric
                except ValueError:
                    pass  # Keep as string if not numeric
                all_dicom_files[file_id] = os.path.join(root, file)
    
    conversion_results = {
        'successful': [],
        'failed': [],
        'not_found': [],
        'total_processed': 0
    }
    
    print(f"Starting conversion of {len(files_to_convert)} DICOM files...")
    print(f"Target monochrome format: {target_monochrome}")
    print(f"Output directory: {EXPERIMENTS_PATH}")
    print()
    
    for file_id in files_to_convert:
        if file_id not in all_dicom_files:
            print(f"File {file_id} not found in DICOM directory")
            conversion_results['not_found'].append(file_id)
            continue
            
        dicom_path = all_dicom_files[file_id]
        
        try:
            # Read DICOM file
            ds = pydicom.dcmread(dicom_path)
            
            # Get pixel data
            pixel_array = ds.pixel_array.astype(np.float64)
            
            # Get photometric interpretation
            photometric = getattr(ds, 'PhotometricInterpretation', 'MONOCHROME2')
            
            # Convert MONOCHROME1 to MONOCHROME2 if needed
            if photometric == 'MONOCHROME1' and target_monochrome == 'MONOCHROME2':
                # In MONOCHROME1, higher values = darker, so invert
                max_possible = np.max(pixel_array)
                pixel_array = max_possible - pixel_array
                print(f"  Converted {file_id} from MONOCHROME1 to MONOCHROME2")
            elif photometric == 'MONOCHROME2' and target_monochrome == 'MONOCHROME1':
                # In MONOCHROME2, higher values = brighter, so invert
                max_possible = np.max(pixel_array)
                pixel_array = max_possible - pixel_array
                print(f"  Converted {file_id} from MONOCHROME2 to MONOCHROME1")
            
            # Get actual bit depth based on pixel values
            min_pixel = np.min(pixel_array)
            max_pixel = np.max(pixel_array)
            
            # Calculate dynamic range
            pixel_range = max_pixel - min_pixel
            
            if pixel_range == 0:
                print(f"  Warning: {file_id} has uniform pixel values")
                # Handle uniform images
                normalized_array = np.zeros_like(pixel_array)
            else:
                # Normalize to 0-1 range
                normalized_array = (pixel_array - min_pixel) / pixel_range
            
            # Scale to RGB range (0 to 255*3 = 765)
            rgb_range = 255 * 3
            scaled_array = normalized_array * rgb_range
            
            # Convert to RGB encoding
            height, width = pixel_array.shape
            rgb_array = np.zeros((height, width, 3), dtype=np.uint8)
            
            # Distribute values across RGB channels
            for i in range(height):
                for j in range(width):
                    value = scaled_array[i, j]
                    
                    # Distribute across R, G, B channels
                    if value <= 255:
                        # All in R channel
                        rgb_array[i, j] = [int(value), 0, 0]
                    elif value <= 510:
                        # R maxed, remainder in G
                        rgb_array[i, j] = [255, int(value - 255), 0]
                    else:
                        # R and G maxed, remainder in B
                        rgb_array[i, j] = [255, 255, int(value - 510)]
            
            # Create PIL Image
            pil_image = Image.fromarray(rgb_array, 'RGB')
            
            # Save as PNG
            output_filename = f"{file_id}_rgb_encoded.png"
            output_path = os.path.join(EXPERIMENTS_PATH, output_filename)
            pil_image.save(output_path)
            
            # Store conversion info
            conversion_info = {
                'file_id': file_id,
                'original_photometric': photometric,
                'target_photometric': target_monochrome,
                'original_range': (min_pixel, max_pixel),
                'dynamic_range': pixel_range,
                'output_file': output_filename
            }
            
            conversion_results['successful'].append(conversion_info)
            conversion_results['total_processed'] += 1
            
            print(f"  ✓ {file_id}: {photometric} → PNG (range: {min_pixel:.0f}-{max_pixel:.0f})")
            
        except Exception as e:
            print(f"  ✗ Error processing {file_id}: {str(e)}")
            conversion_results['failed'].append({'file_id': file_id, 'error': str(e)})
    
    # Print summary
    print("\n=== CONVERSION SUMMARY ===")
    print(f"Successfully converted: {len(conversion_results['successful'])} files")
    print(f"Failed conversions: {len(conversion_results['failed'])} files")
    print(f"Files not found: {len(conversion_results['not_found'])} files")
    print(f"Total processed: {conversion_results['total_processed']} files")
    
    if conversion_results['successful']:
        print(f"\nOutput files saved to: {EXPERIMENTS_PATH}")
        for result in conversion_results['successful']:
            print(f"  - {result['output_file']}")
    
    return conversion_results


### Execution

In [ ]:
# Execute the conversion
results = convert_dicom_to_rgb_png(FILES_TO_CONVERT)

## View highest pixel values in images

### Functions

In [1]:
def generate_pixel_value_mask(file_id, target="max", display=False, export=True):
    """
    Generate a mask showing only the pixels reaching the min or max pixel value in a DICOM file.
    
    Parameters:
    -----------
    file_id : int
        FileID representing the name of the DICOM file without the extension
    target : str, optional
        Target pixel values to mask ("max" or "min"). Default is "max"
    display : bool, optional
        If True, displays the mask and original image in the notebook. Default is False
    export : bool, optional
        If True, exports the mask image to the Experiments folder. Default is True
    
    Returns:
    --------
    dict : Dictionary containing mask statistics and file information
    """
    import matplotlib.pyplot as plt
    
    # Get all DICOM files in the directory
    all_dicom_files = {}
    for root, dirs, files in os.walk(DICOM_PATH):
        for file in files:
            if file.lower().endswith(('.dcm', '.dicom')):
                # Extract FileID (filename without extension)
                file_name = os.path.splitext(file)[0]
                try:
                    file_key = int(file_name)  # Convert to int if it's numeric
                except ValueError:
                    file_key = file_name  # Keep as string if not numeric
                all_dicom_files[file_key] = os.path.join(root, file)
    
    # Check if file exists
    if file_id not in all_dicom_files:
        print(f"Error: File {file_id} not found in DICOM directory")
        return None
    
    dicom_path = all_dicom_files[file_id]
    
    try:
        # Read DICOM file
        ds = pydicom.dcmread(dicom_path)
        pixel_array = ds.pixel_array.astype(np.float64)
        
        # Get min and max pixel values
        min_pixel = np.min(pixel_array)
        max_pixel = np.max(pixel_array)
        
        # Create mask based on target
        if target.lower() == "max":
            mask = (pixel_array == max_pixel)
            target_value = max_pixel
            print(f"Creating mask for maximum pixel value: {max_pixel}")
        elif target.lower() == "min":
            mask = (pixel_array == min_pixel)
            target_value = min_pixel
            print(f"Creating mask for minimum pixel value: {min_pixel}")
        else:
            print(f"Error: Invalid target '{target}'. Use 'max' or 'min'")
            return None
        
        # Count pixels at target value
        pixel_count = np.sum(mask)
        total_pixels = pixel_array.size
        percentage = (pixel_count / total_pixels) * 100
        
        print(f"Found {pixel_count} pixels ({percentage:.2f}%) at {target} value")
        
        # Create mask image (white pixels where condition is met, black elsewhere)
        mask_image = mask.astype(np.uint8) * 255
        
        # Display images if requested
        if display:
            fig, axes = plt.subplots(1, 3, figsize=(15, 5))
            
            # Original image
            axes[0].imshow(pixel_array, cmap='gray')
            axes[0].set_title(f'Original Image (File {file_id})')
            axes[0].axis('off')
            
            # Mask
            axes[1].imshow(mask_image, cmap='gray')
            axes[1].set_title(f'Mask ({target.capitalize()} Value: {target_value})')
            axes[1].axis('off')
            
            # Overlay - Original image with red mask
            # Normalize original image to 0-255 range for RGB display
            normalized_img = ((pixel_array - min_pixel) / (max_pixel - min_pixel) * 255).astype(np.uint8)
            # Create RGB version (grayscale base)
            overlay_rgb = np.stack([normalized_img, normalized_img, normalized_img], axis=-1)
            # Set masked pixels to red
            overlay_rgb[mask] = [255, 0, 0]  # Red color for masked pixels
            axes[2].imshow(overlay_rgb)
            axes[2].set_title('Overlay (Original + Red Mask)')
            axes[2].axis('off')
            
            plt.tight_layout()
            plt.show()
        
        # Export mask if requested
        if export:
            # Create experiments directory if it doesn't exist
            Path(EXPERIMENTS_PATH).mkdir(parents=True, exist_ok=True)
            
            # Save mask as PNG
            mask_pil = Image.fromarray(mask_image, 'L')  # 'L' for grayscale
            output_filename = f"{file_id}_mask_{target}_value.png"
            output_path = os.path.join(EXPERIMENTS_PATH, output_filename)
            mask_pil.save(output_path)
            print(f"Mask exported to: {output_path}")
        
        # Return statistics
        result = {
            'file_id': file_id,
            'target': target,
            'target_value': target_value,
            'pixel_count': int(pixel_count),
            'total_pixels': int(total_pixels),
            'percentage': percentage,
            'min_pixel': min_pixel,
            'max_pixel': max_pixel,
            'mask_shape': mask.shape
        }
        
        if export:
            result['output_file'] = output_filename
        
        return result
        
    except Exception as e:
        print(f"Error processing file {file_id}: {str(e)}")
        return None

### Execution

In [ ]:
# Create a mask for maximum pixel values, display it, and does not export to file
result = generate_pixel_value_mask(28694926, target="max", display=True, export=False)

In [ ]:
# List of images to process
files_to_process = [28694926, 28695053, 28695391, 28695758, 28695894, 28695959, 28695962, 28696435, 28696587, 28696619, 28696622, 28696682, 28696760, 28696772, 28696790, 28696803, 28696818, 28696848, 28696913, 28696923, 28697139, 28697151, 28697157, 28697242, 28697245, 28697266, 28697275, 28697287, 28697301]

# Process all files in the list
results = []
for file_id in files_to_process:
    print(f"Processing file {file_id}...")
    result = generate_pixel_value_mask(file_id, target="max", display=False, export=False)
    if result:
        results.append(result)
        print(f"Successfully processed file {file_id}")
    else:
        print(f"Failed to process file {file_id}")

print(f"\nProcessing complete. Successfully processed {len(results)} out of {len(files_to_process)} files.")